In [2]:
import requests
server_url = "http://localhost:8000"
session_id = '91ab8896-ccbc-4e71-8929-f45855fb15c4'

In [1]:
import json
json.dumps(None)

'null'

In [3]:
import docker

# Initialize the Docker client
client = docker.from_env()

# List all containers
containers = client.containers.list(all=True)

# Find the container whose image name contains 'cityflow_platform'
target_container = None
for container in containers:
    if any('cityflow_platform' in tag for tag in container.image.tags):
        target_container = container
        break

if target_container:
    # Get the container's bind mounts
    bind_mounts = target_container.attrs['HostConfig']['Binds']

    # Find the bind folder for /cityflow_platform/cityflow_executor/code
    for bind in bind_mounts:
        host_path, container_path = bind.split(':')
        if container_path == '/cityflow_platform/cityflow_executor/code':
            print(f"Host path for /cityflow_platform/cityflow_executor/code: {host_path}")
else:
    print("Container with image name containing 'cityflow_platform' not found.")

Host path for /cityflow_platform/cityflow_executor/code: /Users/kai/cityflow_platform/temp


In [5]:
import docker
client = docker.from_env()

In [7]:
container = client.containers.create(
    'ghcr.io/kekehurry/cityflow_runner:light',  # Image name
    # entrypoint="/bin/bash",
    tty=True,
    auto_remove=True,
)

In [4]:
docker.__version__

'7.1.0'

In [21]:
response = requests.post(server_url + "/setup", json={"session_id": session_id, "packages": ["numpy", "pandas"],"language": "python"})
print(response.json())

{'console': "Collecting numpy\n  Downloading numpy-2.2.0-cp313-cp313-manylinux_2_17_aarch64.manylinux2014_aarch64.whl.metadata (63 kB)\nDownloading numpy-2.2.0-cp313-cp313-manylinux_2_17_aarch64.manylinux2014_aarch64.whl (14.0 MB)\n   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 2.6 MB/s eta 0:00:00\nInstalling collected packages: numpy\nSuccessfully installed numpy-2.2.0\nWARNING: Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the system package manager, possibly rendering your system unusable.It is recommended to use a virtual environment instead: https://pip.pypa.io/warnings/venv. Use the --root-user-action option if you know what you are doing and want to suppress this warning.\nCollecting pandas\n  Downloading pandas-2.2.3-cp313-cp313-manylinux2014_aarch64.manylinux_2_17_aarch64.whl.metadata (89 kB)\nRequirement already satisfied: numpy>=1.26.0 in /usr/local/lib/python3.13/site-packages (from pandas) (2.2.0)\nCollecting pyth

In [22]:
code_blocks = [
{
    "code":'''
import numpy as np
print(np.random.rand(5))
''',
"language": "python"
}
]
response = requests.post(server_url + "/execute", json={"code_blocks": code_blocks,"session_id": session_id})
print(response.json())

{'console': '[0.71684392 0.40323118 0.06093366 0.19381175 0.41089875]\n', 'container_name': 'csflow-91ab8896-ccbc-4e71-8929-f45855fb15c4', 'exit_code': 0, 'output': ''}


In [23]:
code_blocks = [
{
    "code":'''
with open("input") as f:
    a = f.read()
    print(a)
with open("output", "w") as f:
    f.write(a+"hahaha")
''',
"language": "python",
"files": [{
    "name": "input",
    "content": "Hello, World!"
}]
}
]
response = requests.post(server_url + "/execute", json={"code_blocks": code_blocks, "session_id": session_id})
print(response.json())

{'console': 'Hello, World!\n', 'container_name': 'csflow-91ab8896-ccbc-4e71-8929-f45855fb15c4', 'exit_code': 0, 'output': 'Hello, World!hahaha'}


In [24]:
response = requests.post(server_url + "/keep_alive", json={"session_id": session_id})
print(response.json())

{'container_name': 'csflow-91ab8896-ccbc-4e71-8929-f45855fb15c4', 'last_update': 1733710282.177977}


In [25]:
response = requests.post(server_url + "/kill", json={"session_id": session_id})
print(response.json())

{'container_name': 'csflow-91ab8896-ccbc-4e71-8929-f45855fb15c4', 'exit_code': 0, 'output': 'Container has been removed.'}
